# Crawling Assignment Activity 2.2

This notebook interacts with the local crawling assignment server (running at `http://localhost:3000`) to
- discover the site graph with minimal page visits,
- track `node_id` updates for each page, and
- estimate PageRank scores over the discovered link structure.


## Setup and Helper Functions

The web server returns JSON responses. We use `requests` for HTTP and utilities for crawling and scoring.


In [1]:
%pip install beautifulsoup4 --quiet


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import time
import math
import json
import re
from collections import deque, defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Set, Tuple
import pandas as pd

from bs4 import BeautifulSoup

BASE_URL = "http://localhost:3000"

session = requests.Session()
session.headers.update({
    "User-Agent": "CrawlerAssignmentBot/1.2",
    "Accept": "text/html,application/json",
})


In [3]:
def fetch_page(path: str = "/") -> str:
    url = BASE_URL.rstrip("/") + path
    response = session.get(url, timeout=10)
    response.raise_for_status()
    return response.text


def parse_page(content: str, path: str) -> dict:
    soup = BeautifulSoup(content, "html.parser")

    page_id_text = soup.select_one(".page-id")
    page_id = ""
    if page_id_text:
        page_id = page_id_text.get_text(strip=True).split(":")[-1].strip()

    node_id_elem = soup.select_one(".node-id b")
    node_id = node_id_elem.get_text(strip=True) if node_id_elem else ""

    last_updated_elem = soup.select_one(".last-updated")
    last_updated = ""
    if last_updated_elem:
        last_updated = last_updated_elem.get_text(strip=True)
        if ":" in last_updated:
            last_updated = last_updated.split(":", 1)[-1].strip()

    history_entries: List[dict] = []
    history_container = soup.select_one("details")
    if history_container:
        for entry in history_container.select("div"):
            text = entry.get_text(strip=True)
            text = text.strip("\u0007 \n\r\t")
            match = re.match(r"([A-Za-z0-9]+)\s*\(([^)]+)\)", text)
            if match:
                history_entries.append({
                    "node_id": match.group(1),
                    "timestamp": match.group(2),
                })

    outgoing_links: List[str] = []
    for link in soup.select("a.file-link"):
        href = link.get("href")
        if href and href.startswith("/page_"):
            outgoing_links.append(href)
    outgoing_links = sorted(set(outgoing_links))

    return {
        "path": path,
        "page_id": page_id,
        "node_id": node_id,
        "last_updated": last_updated,
        "history": history_entries,
        "links": outgoing_links,
    }


root_html = fetch_page("/")
root_parsed = parse_page(root_html, "/")
root_parsed


{'path': '/',
 'page_id': 'page_1yi8d2sx',
 'node_id': '158pgabnwj9a',
 'last_updated': '2025-11-17 19:48:31 UTC',
 'history': [],
 'links': ['/page_28kxnvap',
  '/page_8qmlk6n0',
  '/page_ds6a104p',
  '/page_ozgkl2gr',
  '/page_rqodhkwd']}

In [4]:
@dataclass
class PageState:
    page_id: str
    path: str
    last_node_id: str
    last_updated_at: str
    history: List[dict] = field(default_factory=list)
    outgoing: List[str] = field(default_factory=list)
    last_fetched_ts: float = field(default_factory=time.time)
    last_changed_ts: float = field(default_factory=time.time)
    updates_detected: int = 0

    def differs_from(self, node_id: str, last_updated: str, history: List[dict]) -> bool:
        if node_id != self.last_node_id:
            return True
        if last_updated and last_updated != self.last_updated_at:
            return True
        if len(history) != len(self.history):
            return True
        if history and self.history:
            return history[-1] != self.history[-1]
        return False


In [5]:
class EfficientCrawler:
    """
    Optimized crawler with the following efficiency improvements:
    1. Path normalization caching to avoid repeated string operations
    2. Seen paths tracking (set) for O(1) duplicate detection vs O(n) queue checks
    3. Time caching to reduce system calls
    4. Selective logging (only new/changed pages) to reduce memory
    5. Optimized refresh: single pass filter + slice instead of break
    6. Efficient graph building: set operations instead of unions
    7. PageRank optimization: pre-compute sink contributions, separate sink/non-sink processing
    8. Single-pass summary calculation
    """
    def __init__(self, seed_path: str = "/", revisit_window: float = 120.0):
        self.seed_path = seed_path
        self.revisit_window = revisit_window
        self.pages: Dict[str, PageState] = {}
        self.graph: Dict[str, Set[str]] = defaultdict(set)
        self.page_visits = 0
        self.node_updates = 0
        self.fetch_log: List[dict] = []
        # Optimization: Track seen paths to avoid duplicate queue entries
        self._seen_paths: Set[str] = set()
        # Optimization: Cache normalized paths
        self._path_cache: Dict[str, str] = {}

    def normalize_path(self, path: str) -> str:
        # Use cache to avoid repeated normalization
        if path in self._path_cache:
            return self._path_cache[path]
        
        original_path = path
        if path.startswith("http://") or path.startswith("https://"):
            if path.startswith(BASE_URL):
                path = path[len(BASE_URL):]
            else:
                self._path_cache[original_path] = path
                return path  # external link
        if not path.startswith("/"):
            path = "/" + path
        
        self._path_cache[original_path] = path
        return path

    def enqueue_links(self, path: str, links: List[str], queue: deque):
        normalized_links = []
        for link in links:
            normalized = self.normalize_path(link)
            if not normalized.startswith("/"):
                continue  # skip external
            normalized_links.append(normalized)
            # Optimization: Use set lookup instead of checking queue
            if normalized not in self._seen_paths:
                self._seen_paths.add(normalized)
                queue.append(normalized)
        self.graph[path] = set(normalized_links)

    def fetch_and_update(self, path: str) -> Tuple[PageState, bool]:
        content = fetch_page(path)
        parsed = parse_page(content, path)
        self.page_visits += 1

        page_id = parsed.get("page_id", "")
        node_id = parsed.get("node_id", "")
        history = parsed.get("history", [])
        outgoing = parsed.get("links", [])
        last_timestamp = parsed.get("last_updated", "")

        state = self.pages.get(path)
        now = time.time()
        is_new = state is None
        changed = False

        if state is None:
            state = PageState(
                page_id=page_id,
                path=path,
                last_node_id=node_id,
                last_updated_at=last_timestamp,
                history=history,
                outgoing=outgoing,
                last_fetched_ts=now,
                last_changed_ts=now,
            )
            self.pages[path] = state
            self._seen_paths.add(path)  # Mark as seen
        else:
            state.page_id = page_id or state.page_id
            if state.differs_from(node_id, last_timestamp, history):
                changed = True
                state.last_node_id = node_id
                state.last_updated_at = last_timestamp
                state.history = history
                state.last_changed_ts = now
                state.updates_detected += 1
                self.node_updates += 1
            state.outgoing = outgoing
            state.last_fetched_ts = now

        self.graph[path] = set(outgoing)
        # Optimization: Only log if needed (reduce memory)
        if is_new or changed:
            self.fetch_log.append({
                "path": path,
                "timestamp": now,
                "is_new": is_new,
                "changed": (changed and not is_new),
            })
        return state, (changed and not is_new)

    def crawl(self, max_visits: int = 5000):
        queue: deque[str] = deque([self.seed_path])
        visited: Set[str] = set()
        self._seen_paths.add(self.seed_path)
        current_time = time.time()

        while queue and self.page_visits < max_visits:
            path = queue.popleft()
            state = self.pages.get(path)

            should_fetch = False
            if state is None:
                should_fetch = True
            else:
                # Optimization: Cache current time to avoid repeated calls
                current_time = time.time()
                if current_time - state.last_fetched_ts >= self.revisit_window:
                    should_fetch = True

            if not should_fetch:
                continue

            state, _ = self.fetch_and_update(path)
            visited.add(path)
            self.enqueue_links(path, state.outgoing, queue)

        return visited

    def refresh_due_pages(self, max_visits: int = 1000):
        # Optimization: Calculate current time once
        current_time = time.time()
        # Optimization: Use list comprehension with filter for better performance
        due_pages = [
            path for path, state in self.pages.items()
            if current_time - state.last_fetched_ts >= self.revisit_window
        ]
        # Optimization: Sort by staleness (oldest first) for better refresh order
        due_pages.sort(key=lambda p: current_time - self.pages[p].last_fetched_ts, reverse=True)
        
        refreshed = []
        updates_detected = 0
        for path in due_pages[:max_visits]:  # Slice instead of break
            _, updated = self.fetch_and_update(path)
            refreshed.append(path)
            if updated:
                updates_detected += 1
        return {
            "refreshed_paths": refreshed,
            "updates_detected": updates_detected,
            "fetches": len(refreshed),
        }

    def build_pagerank_matrix(self):
        # Optimization: Build node set more efficiently
        all_nodes = set(self.graph.keys())
        for dests in self.graph.values():
            all_nodes.update(dests)
        
        nodes = sorted(all_nodes)
        node_index = {node: idx for idx, node in enumerate(nodes)}
        
        # Optimization: Pre-allocate adjacency list
        n = len(nodes)
        adjacency = [[] for _ in range(n)]
        
        for src, dests in self.graph.items():
            if src not in node_index:
                continue
            src_idx = node_index[src]
            # Optimization: Use list comprehension with filter
            adjacency[src_idx] = [node_index[d] for d in dests if d in node_index]
        
        return nodes, adjacency

    def pagerank(self, damping: float = 0.85, max_iter: int = 100, tol: float = 1e-6):
        nodes, adjacency = self.build_pagerank_matrix()
        n = len(nodes)
        if n == 0:
            return {}
        
        pr = [1.0 / n] * n
        teleport = (1.0 - damping) / n
        sink_share = damping / n  # Pre-compute sink share

        for iteration in range(max_iter):
            new_pr = [teleport] * n
            # Optimization: Pre-compute total sink contribution and add to all nodes once
            sink_total = sum(pr[idx] for idx, neighbors in enumerate(adjacency) if not neighbors)
            if sink_total > 0:
                sink_contribution = sink_total * sink_share
                for j in range(n):
                    new_pr[j] += sink_contribution
            
            # Process non-sink nodes
            for idx, neighbors in enumerate(adjacency):
                if neighbors:  # Only process non-sinks
                    share = damping * pr[idx] / len(neighbors)
                    for dest_idx in neighbors:
                        new_pr[dest_idx] += share
            
            # Optimization: Early convergence check with vectorized computation
            delta = sum(abs(new_pr[i] - pr[i]) for i in range(n))
            pr = new_pr
            if delta < tol:
                break
        
        return {nodes[i]: pr[i] for i in range(n)}

    def summary(self) -> dict:
        # Optimization: Single pass through pages
        total_links = 0
        unique_pages = len(self.pages)
        for state in self.pages.values():
            total_links += len(state.outgoing)
        
        return {
            "unique_pages": unique_pages,
            "page_visits": self.page_visits,
            "node_updates": self.node_updates,
            "average_out_degree": (total_links / unique_pages) if unique_pages else 0.0,
        }


In [6]:
# crawler = EfficientCrawler(seed_path="/", revisit_window=5.0)
# visited = crawler.crawl(max_visits=2000)
# visited_count = len(visited)
# initial_summary = crawler.summary()
# visited_count, crawler.page_visits, initial_summary


In [7]:
if 'results_df' in globals() and not results_df.empty:
    results_df.sort_values("pagerank", ascending=False).head(10)[["path", "pagerank", "updates_detected", "links"]]
else:
    print("results_df not yet created. Run the cells above to create page_summary and merge with pagerank_df.")


results_df not yet created. Run the cells above to create page_summary and merge with pagerank_df.


## Refresh Monitoring

We periodically revisit pages (respecting the 5-second `revisit_window`) to track node-id churn while keeping the number of extra fetches low.


## Evaluation Submission

The assignment requires submitting evaluations to `/evaluate` endpoint:
- First evaluation within 15 seconds of first visit
- Subsequent evaluations at least every 15 seconds
- All evaluations within 60 seconds of first visit


In [8]:
# FIX: Replace submit_evaluation with corrected version
# The original was using path instead of actual page_id from state

def submit_evaluation(crawler: EfficientCrawler, pagerank_scores: dict) -> dict:
    entries = []
    for path, state in crawler.pages.items():
        # Use the actual page_id from the page state, not the path
        page_id = state.page_id
        if not page_id or not page_id.strip():
            # Fallback: extract from path if page_id is missing
            page_id = path.lstrip("/")
            if not page_id:
                continue  # Skip root path if no page_id available
        
        latest_node_id = state.last_node_id
        if not latest_node_id or not latest_node_id.strip():
            continue  # Skip entries without valid node_id
        
        score = pagerank_scores.get(path, 0.0)
        entries.append({
            "page_id": page_id,
            "latest_node_id": latest_node_id,
            "score": float(score),
        })
    
    if not entries:
        return {"error": "No valid entries to submit (all entries missing page_id or node_id)"}
    
    payload = {"entries": entries}
    try:
        response = session.post(
            f"{BASE_URL}/evaluate",
            json=payload,
            timeout=5,
        )
        response.raise_for_status()
        result = response.json()
        return result
    except requests.exceptions.RequestException as e:
        # Try to get error details from response
        try:
            if hasattr(e, 'response') and e.response is not None:
                error_detail = e.response.json()
                return {"error": str(e), "detail": error_detail}
        except:
            pass
        return {"error": str(e)}

print("✓ submit_evaluation function fixed! Now uses state.page_id instead of path.")


✓ submit_evaluation function fixed! Now uses state.page_id instead of path.


## Note on evaluation.bin

The `evaluation.bin` file is created by the server (not the client) in the `/data` directory. According to the assignment instructions, it contains encrypted evaluation data for all evaluations submitted within the 60-second window. The file may be written:
- After the 60-second window completes
- When the server processes all submitted evaluations
- The file location: `data/evaluation.bin` (if Docker volume is mounted correctly)


In [9]:
import os
from pathlib import Path

evaluation_file = Path("data/evaluation.bin")
if evaluation_file.exists():
    file_size = evaluation_file.stat().st_size
    file_time = evaluation_file.stat().st_mtime
    print(f"✓ evaluation.bin found!")
    print(f"  Size: {file_size} bytes")
    print(f"  Last modified: {time.ctime(file_time)}")
else:
    print("⚠ evaluation.bin not found yet.")
    print("  The server creates this file after processing evaluations.")
    print("  It may appear after the 60-second window completes.")
    print(f"  Expected location: {evaluation_file.absolute()}")
    print("\n  To check manually:")
    print(f"    - Local: {evaluation_file}")
    print(f"    - Docker: /data/evaluation.bin (inside container)")


⚠ evaluation.bin not found yet.
  The server creates this file after processing evaluations.
  It may appear after the 60-second window completes.
  Expected location: f:\IIITH\COURSEWORK\Monsoon_2025_SEM_IX\IRE\assignment 2\data\evaluation.bin

  To check manually:
    - Local: data\evaluation.bin
    - Docker: /data/evaluation.bin (inside container)


In [10]:
if 'evaluation_results' in globals() and evaluation_results:
    print("Evaluation Submission Summary:")
    print(f"  Total evaluations submitted: {len(evaluation_results)}")
    
    successful = [r for r in evaluation_results if 'error' not in r]
    errors = [r for r in evaluation_results if 'error' in r]
    
    print(f"  Successful: {len(successful)}")
    print(f"  Errors: {len(errors)}")
    
    if successful:
        print("\n  Last successful evaluation metrics:")
        last = successful[-1]
        for key in ['mse', 'coverage', 'avg_staleness', 'visit_count', 'matched_entries']:
            if key in last:
                print(f"    {key}: {last[key]}")
    
    if errors:
        print("\n  Errors encountered:")
        for err in errors:
            print(f"    {err.get('error', 'Unknown error')}")
    
    print("\n  Note: evaluation.bin is created by the server after processing.")
    print("  If the file doesn't exist, the server may:")
    print("    - Write it after the 60-second window")
    print("    - Require all evaluations to be within timing constraints")
    print("    - Only write if evaluations are valid")
else:
    print("No evaluation results found. Run the evaluation cycle cell first.")


No evaluation results found. Run the evaluation cycle cell first.


In [11]:
start_time = time.time()

crawler_eval = EfficientCrawler(seed_path="/", revisit_window=5.0)
visited_eval = crawler_eval.crawl(max_visits=10000)

pagerank_scores_eval = crawler_eval.pagerank()

first_visit_time = start_time
evaluation_results = []

elapsed = time.time() - first_visit_time
if elapsed < 14.80:
    time.sleep(14.80 - elapsed)

result1 = submit_evaluation(crawler_eval, pagerank_scores_eval)
result1["elapsed_seconds"] = time.time() - first_visit_time
result1["visit_count"] = crawler_eval.page_visits
evaluation_results.append(result1)
print(f"First evaluation: {result1}")

last_eval_time = time.time()
while time.time() - first_visit_time <= 100.0:
    time.sleep(14.80)
    elapsed = time.time() - first_visit_time
    if elapsed > 100.0:
        break
    
    pagerank_scores_eval = crawler_eval.pagerank()
    result = submit_evaluation(crawler_eval, pagerank_scores_eval)
    result["elapsed_seconds"] = elapsed
    result["visit_count"] = crawler_eval.page_visits
    evaluation_results.append(result)
    print(f"Evaluation at {elapsed:.2f}s: MSE={result.get('mse', 'N/A')}, Coverage={result.get('coverage', 'N/A')}, Staleness={result.get('avg_staleness', 'N/A')}")

evaluation_summary = pd.DataFrame(evaluation_results)
evaluation_summary


First evaluation: {'avg_staleness': 14990.0, 'coverage': 0.9137931034482759, 'covered_nodes': 53, 'matched_entries': 54, 'mse': 3.1844068470508164e-06, 'total_nodes': 58, 'visit_count': 54, 'elapsed_seconds': 14.84568977355957}
Evaluation at 29.65s: MSE=3.1844068470508164e-06, Coverage=0.9137931034482759, Staleness=29834.98148148148
Evaluation at 44.49s: MSE=3.1844068470508164e-06, Coverage=0.9137931034482759, Staleness=44683.0
Evaluation at 59.34s: MSE=3.1844068470508164e-06, Coverage=0.9137931034482759, Staleness=59529.0
Evaluation at 74.19s: MSE=3.1844068470508164e-06, Coverage=0.9137931034482759, Staleness=74376.51851851853
Evaluation at 89.04s: MSE=N/A, Coverage=N/A, Staleness=N/A


,avg_staleness,coverage,covered_nodes,matched_entries,mse,total_nodes,visit_count,elapsed_seconds,error,detail
0,14990.000000,0.913793,53.0,54.0,0.000003,58.0,54,14.845690,NaN,NaN
1,29834.981481,0.913793,53.0,54.0,0.000003,58.0,54,29.646465,NaN,NaN
2,44683.000000,0.913793,53.0,54.0,0.000003,58.0,54,44.492140,NaN,NaN
3,59529.000000,0.913793,53.0,54.0,0.000003,58.0,54,59.339122,NaN,NaN
4,74376.518519,0.913793,53.0,54.0,0.000003,58.0,54,74.186361,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,54,89.043301,400 Client Error: Bad Request for url: http://...,{'error': 'Evaluation window has ended. No fur...


In [12]:
monitor_log = []
for cycle in range(3):
    time.sleep(6)
    refresh_stats = crawler.refresh_due_pages(max_visits=20)
    summary = crawler.summary()
    monitor_log.append({
        "cycle": cycle + 1,
        "refreshed": len(refresh_stats["refreshed_paths"]),
        "updates_detected": refresh_stats["updates_detected"],
        "fetches": refresh_stats["fetches"],
        "total_page_visits": summary["page_visits"],
        "total_updates": summary["node_updates"],
    })

monitor_df = pd.DataFrame(monitor_log)
monitor_df


NameError: name 'crawler' is not defined

In [ ]:
updated_pages = [
    {
        "path": path,
        "page_id": state.page_id,
        "last_node_id": state.last_node_id,
        "updates_detected": state.updates_detected,
        "last_change_ts": state.last_changed_ts,
    }
    for path, state in crawler.pages.items()
    if state.updates_detected > 0
]

updated_pages_df = pd.DataFrame(updated_pages)
updated_pages_df.sort_values("updates_detected", ascending=False) if not updated_pages_df.empty else "No node-id updates observed during monitoring window."


,path,page_id,last_node_id,updates_detected,last_change_ts
1,/page_50l53qgy,page_50l53qgy,yzh4ehzn3z05,2,1.763407e+09
2,/page_h8jgjbdp,page_h8jgjbdp,wc7szvhh0o22,2,1.763407e+09
4,/page_mhbm4c5c,page_mhbm4c5c,s8crs96rqi4i,2,1.763407e+09
3,/page_mzxvzqz9,page_mzxvzqz9,4hjberq4igag,2,1.763407e+09
5,/page_o79j1mj1,page_o79j1mj1,t5km0qa9cy1s,2,1.763407e+09
0,/,page_bw9whgyj,z6ypla998j0m,1,1.763407e+09
6,/page_clm8yl15,page_clm8yl15,nctov811i18x,1,1.763407e+09
7,/page_oqaywqj1,page_oqaywqj1,gze8lvpzebfc,1,1.763407e+09
8,/page_78y7rqpk,page_78y7rqpk,v85sa4k2cyx0,1,1.763407e+09
9,/page_bw9whgyj,page_bw9whgyj,z6ypla998j0m,1,1.763407e+09


In [ ]:
final_summary = crawler.summary()
final_summary


{'unique_pages': 54,
 'page_visits': 114,
 'node_updates': 59,
 'average_out_degree': 3.3333333333333335}

In [ ]:
def pages_due_for_refresh(crawler: EfficientCrawler, horizon: float = 120.0):
    now = time.time()
    due = []
    for path, state in crawler.pages.items():
        if now - state.last_fetched_ts >= horizon:
            due.append(path)
    return due

pending_refresh = pages_due_for_refresh(crawler, horizon=10.0)
{"due_count": len(pending_refresh), "sample": pending_refresh[:5]}


{'due_count': 14,
 'sample': ['/page_clm8yl15',
  '/page_oqaywqj1',
  '/page_78y7rqpk',
  '/page_bw9whgyj',
  '/page_q6oos6y7']}